<a href="https://colab.research.google.com/github/Kkordik/NovelQSI/blob/main/TriviaQA_to_SQuAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is how to convert TriviaQA dataset to the squad format

I will be using:
https://github.com/mandarjoshi90/triviaqa


NOTE, that just running all the cells is not enough, you will need to edit the /content/triviaqa/utils/convert_to_squad_format.py and /content/triviaqa/utils/dataset_utils.py files to let it work.

Also before editing check the issue I've opened at github, maybe the problems have been already solved, at the same page I've described how to solve the problems if no changes in github's code were done:
https://github.com/mandarjoshi90/triviaqa/issues/9

In [19]:
# @title Install TriviaQA dataset and extract all the dataset files from tar
!wget https://nlp.cs.washington.edu/triviaqa/data/triviaqa-rc.tar.gz
!tar -xf triviaqa-rc.tar.gz

--2023-12-02 10:50:53--  https://nlp.cs.washington.edu/triviaqa/data/triviaqa-rc.tar.gz
Resolving nlp.cs.washington.edu (nlp.cs.washington.edu)... 128.208.3.120, 2607:4000:200:12::78
Connecting to nlp.cs.washington.edu (nlp.cs.washington.edu)|128.208.3.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2665779500 (2.5G) [application/x-gzip]
Saving to: ‘triviaqa-rc.tar.gz’

triviaqa-rc.tar.gz  100%[===================>]   2.48G  19.7MB/s    in 92s     

2023-12-02 10:52:25 (27.7 MB/s) - ‘triviaqa-rc.tar.gz’ saved [2665779500/2665779500]



In [20]:
# @title Install code from github triviaqa
!git clone https://github.com/mandarjoshi90/triviaqa
%cd /content/triviaqa

Cloning into 'triviaqa'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 67
Receiving objects: 100% (70/70), 21.55 KiB | 3.08 MiB/s, done.
Resolving deltas: 100% (26/26), done.
/content/triviaqa


In [53]:
# @title Run the converting
import nltk
nltk.download('punkt')

triviaqa_file = "/content/triviaqa/qa/wikipedia-dev.json" # @param {type:"string"}
squad_file = "/content/triviaqa/qa/wikipedia-dev-squad1.json" # @param {type:"string"}
result_sqad_file = "/content/dataset_trivia_squad/trivia_qa_squad1.jsonl"
max_num_tokens = 5000 # @param

wikipedia_dir = "/content/triviaqa/evidence/wikipedia"
web_dir = "/content/triviaqa/evidence/web"

!python /content/triviaqa/convert_to_squad_format.py --triviaqa_file $triviaqa_file --squad_file $squad_file --wikipedia_dir $wikipedia_dir --web_dir $web_dir --max_num_tokens $max_num_tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


usage: convert_to_squad_format.py [-h] [--triviaqa_file TRIVIAQA_FILE] [--squad_file SQUAD_FILE]
                                  [--wikipedia_dir WIKIPEDIA_DIR] [--web_dir WEB_DIR]
                                  [--seed SEED] [--max_num_tokens MAX_NUM_TOKENS]
                                  [--sample_size SAMPLE_SIZE] [--tokenizer TOKENIZER]
convert_to_squad_format.py: error: argument --triviaqa_file: expected one argument


In [46]:
# @title Final reformating to real squad format
import json

# Read the JSON file
with open(squad_file, 'r', encoding='utf-8') as file:
    data = json.load(file)

with open(result_sqad_file, 'a', encoding='utf-8') as fa:
  for line in data['data']:
    new_line = {
        "context": line["paragraphs"][0]["context"],
        "id": line["paragraphs"][0]["qas"][0]["id"],
        "answers": {
            "answer_start": [answer["answer_start"] for answer in line["paragraphs"][0]["qas"][0]["answers"]],
            "text": [answer["text"] for answer in line["paragraphs"][0]["qas"][0]["answers"]]
            },
        "question": line["paragraphs"][0]["qas"][0]["question"]
    }
    fa.write(json.dumps(new_line) + '\n')